In [1]:
import datetime as dt
import json
from pprint import pprint

import pandas as pd
import requests

import config

In [2]:
# pd.set_option("display.max_rows", None)
# pd.reset_option("display.max_rows")

In [3]:
WORKSPACE_ID = config.WORKSPACE_ID
USER_ID = config.USER_ID
TOKEN_VALUE = config.TOKEN_VALUE
API_KEY = config.API_KEY
API_SECRET = config.API_SECRET

In [4]:
headers = {
    "workspaceId": WORKSPACE_ID,
    "userId": USER_ID,
    "tokenValue": TOKEN_VALUE,
    "apiKey": API_KEY,
    "apiSecret": API_SECRET,
}

# Get location heirarchy

In [5]:
url = "https://server.itemit.com/collections/itemit/v2.0/locations/hierarchy"

payload = {}

response = requests.request("GET", url, headers=headers, data=payload)

In [6]:
locations = response.json()

In [7]:
pprint(locations[0]["nodes"])

[{'location': {'canAddDuplicateItems': False,
               'collectionType': 'LOCATION',
               'colour': '#1976D2',
               'created': {'_user': {'id': '944e89dc-e4b8-6854-44bb-0000ce3064bd'},
                           'client': {'apiVersion': 4.0, 'platform': 'WEB'},
                           'datetime': '2020-10-26T10:51:51.118Z'},
               'id': 'ab1c55db-5efc-40f1-a3e3-518a413da948',
               'lastSeen': {'_user': {'email': 'cu.covid.testing.4@gmail.com',
                                      'firstName': 'Covid',
                                      'id': 'ccc53756-4506-4685-934d-09f92d8e6cef',
                                      'lastName': 'Testing',
                                      'revision': 1,
                                      'type': 'USER'},
                            'address': 'Unnamed Road, Great Shelford, '
                                       'Cambridge CB22, UK',
                            'coordinates': {'latitude': 52

In [8]:
location_list = []
for location in locations:
    location_list.append(location["location"]["name"])
    if location["nodes"]:
        for node in location["nodes"]:
            location_list.append(node["location"]["name"])
# location_list

In [9]:
loc_map = {
    "*West-Forvie Building": "WFB",
    "Box Storage Room": "WFB",
    "Goods In": "WFB",
    "Overflow": "WFB",
    "Production Lines": "WFB",
    "Anne McLaren Building": "AMB",
    "CUED Pod": None,
    "Christs College": "Colleges",
    "Churchill College": "Colleges",
    "Clare College": "Colleges",
    "Clare Colony": "Colleges",
    "Clare Hall": "Colleges",
    "Corpus Christi Main Plodge": "Colleges",
    "Corpus Leckhampton": "Colleges",
    "Damaged": None,
    "Darwin College": "Colleges",
    "Downing Main Plodge": "Colleges",
    "Drop-Off Pods": "DO",
    "Engineering Pod": "DO",
    "Homerton Pod": "DO",
    "Jesus Pod": "DO",
    "Johns Pod": "DO",
    "Newnham Pod": "DO",
    "Wychfield Pod": "DO",
    "Emmanuel Main Plodge": "Colleges",
    "Fitzwilliam College": "Colleges",
    "Girton College": "Colleges",
    "Gonville & Caius College": "Colleges",
    "Homerton Main Plodge": "Colleges",
    "Homterton Pod": None,
    "Hughes Hall": "Colleges",
    "In Transit": "UMS",
    "In-Transit Paul": "UMS",
    "Jesus College": "Colleges",
    "Kings Main Plodge": "Colleges",
    "Lost": None,
    "Lucy Cavendish": "Colleges",
    "MBIT": None,
    "Magdalene College": "Colleges",
    "Murray Edwards College": "Colleges",
    "Newnham College": "Colleges",
    "Pembroke Main Plodge": "Colleges",
    "Peterhouse Main Plodge": "Colleges",
    "Queens Owlstone Croft": "Colleges",
    "Queens' College": "Colleges",
    "Ridley Hall": "Colleges",
    "Robinson College": "Colleges",
    "Selwyn College": "Colleges",
    "Sidney Sussex College": "Colleges",
    "St Catherine’s Main Plodge": "Colleges",
    "St Catherine’s St Chads": "Colleges",
    "St Edmunds College": "Colleges",
    "St Johns College": "Colleges",
    "Test Site": None,
    "Trinity College": "Colleges",
    "Trinity Hall": "Colleges",
    "Wesley House": "Colleges",
    "Westcot House": "Colleges",
    "Westfield House": "Colleges",
    "Westminster College": "Colleges",
    "Wolfson College": "Colleges",
}

assert sorted(location_list) == sorted(
    loc_map.keys()
), "Location map values do not match heirarchy from API"

## Get number of box items

In [10]:
url = "https://server.itemit.com/items/itemit/v4/count"

filters = {
    "allOf": [
        {
            "anyOf": [
                {
                    "where": "COLLECTION",
                    "has": {"name": "Drop-Off Point Box"},
                },
                {
                    "where": "COLLECTION",
                    "has": {"name": "Delivery Box - Red"},
                },
            ]
        },
        {"allOf": []},
        {"noneOf": []},
    ]
}

payload = {
    "search": "",
    "filters": filters,
    "sorts": [],
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

In [11]:
size = response.json()
assert type(size) == int, "size response should be of type int"

In [12]:
print(size)

195


## Get all box items

In [13]:
url = "https://server.itemit.com/items/itemit/v7/profiles/_search"

sorts = [{"sort": "ITEM", "by": {"name": "ASC"}}]

payload = {
    "size": size,
    "page": 1,
    "search": "",
    "filters": filters,
    "sorts": sorts,
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

In [14]:
items = response.json()

In [15]:
pprint(items[0])

{'_deprecated_': False,
 '_expandedDate': '2021-03-15T09:51:20.568Z',
 '_source': {'created': {'_user': {'id': '944e89dc-e4b8-6854-44bb-0000ce3064bd'},
                         'client': {'apiVersion': 1.0, 'platform': 'SERVER'},
                         'datetime': '2020-09-25T14:56:14.071Z'},
             'description': None,
             'id': 'dfd29d7d-aef2-494d-8ae1-1607d4b97531',
             'lastSeen': {'_user': {'email': 'cu.covid.testing.4@gmail.com',
                                    'firstName': 'Covid',
                                    'id': 'ccc53756-4506-4685-934d-09f92d8e6cef',
                                    'lastName': 'Testing',
                                    'revision': 1,
                                    'type': 'USER'},
                          'address': 'Location not captured',
                          'coordinates': None,
                          'datetime': '2021-03-10T13:07:38.250Z',
                          'method': 'SCAN_QR'},
        

In [16]:
item = items[0]

pprint(item)

{'_deprecated_': False,
 '_expandedDate': '2021-03-15T09:51:20.568Z',
 '_source': {'created': {'_user': {'id': '944e89dc-e4b8-6854-44bb-0000ce3064bd'},
                         'client': {'apiVersion': 1.0, 'platform': 'SERVER'},
                         'datetime': '2020-09-25T14:56:14.071Z'},
             'description': None,
             'id': 'dfd29d7d-aef2-494d-8ae1-1607d4b97531',
             'lastSeen': {'_user': {'email': 'cu.covid.testing.4@gmail.com',
                                    'firstName': 'Covid',
                                    'id': 'ccc53756-4506-4685-934d-09f92d8e6cef',
                                    'lastName': 'Testing',
                                    'revision': 1,
                                    'type': 'USER'},
                          'address': 'Location not captured',
                          'coordinates': None,
                          'datetime': '2021-03-10T13:07:38.250Z',
                          'method': 'SCAN_QR'},
        

In [17]:
print("Current locations of all Drop-Off Point Boxes\n".upper())
print(
    f"No.\t{'Box':12s}\t{'Collection':12s}\t{'Location':22s}\t{'Last Seen Time':20s}\t{'Last Seen By':20s}"
)
print(
    f"---\t{'----':12s}\t{'--------':12s}\t{'--------------':20s}\t{'------------':20s}"
)

data = {
    "box": [],
    "collection": [],
    "location": [],
    "last_seen_dt": [],
    "last_seen_by": [],
    "last_seen_by_email": [],
}

for i, item in enumerate(items):
    box = item["_source"]["name"]
    collection = [
        element["_source"]["name"]
        for element in item["parentObjects"]["elements"]
        if element["_source"]["collectionType"] == "COLLECTION"
    ]
    collection = next(iter(collection), None)

    location = [
        element["_source"]["name"]
        for element in item["parentObjects"]["elements"]
        if element["_source"]["collectionType"] == "LOCATION"
    ]
    location = next(iter(location), None)

    try:
        last_seen_time = item["_source"]["lastSeen"]["datetime"]
        last_seen_time = pd.to_datetime(last_seen_time)
    except TypeError:
        last_seen_time = None

    try:
        first_name = item["_source"]["lastSeen"]["_user"]["firstName"]
    except TypeError:
        first_name = ""
    try:
        last_name = item["_source"]["lastSeen"]["_user"]["lastName"]
    except TypeError:
        last_name = ""
    last_seen_by = f"{first_name} {last_name}"
    last_seen_by = last_seen_by if last_seen_by != " " else None

    try:
        last_seen_by_email = item["_source"]["lastSeen"]["_user"]["email"]
    except TypeError:
        last_seen_by_email = None

    data["box"].append(box)
    data["collection"].append(collection)
    data["location"].append(location)
    data["last_seen_dt"].append(last_seen_time)
    data["last_seen_by"].append(last_seen_by)
    data["last_seen_by_email"].append(last_seen_by_email)

    print(
        f"{i:2d}.\t{data['box'][i]:12.12s}\t{str(data['collection'][i]):12.12s}\t{str(data['location'][i]):22.22s}\t{str(data['last_seen_dt'][i]):20.20s}\t{str(data['last_seen_by'][i]):20.20s}"
        #         f"{i:2d}.\t{data['box'][i]:12.12s}\t{data['location'][i]:22.22s}\t{data['last_seen_dt'][i]:20.20s}\t{data['last_seen_by'][i]:20.20s}"
    )

CURRENT LOCATIONS OF ALL DROP-OFF POINT BOXES

No.	Box         	Collection  	Location              	Last Seen Time      	Last Seen By        
---	----        	--------    	--------------      	------------        
 0.	CAI-1-A     	Delivery Box	Gonville & Caius Colle	2021-03-10 13:07:38.	Covid Testing       
 1.	CAI-1-B     	Delivery Box	Box Storage Room      	2021-03-11 09:41:30.	Covid Testing       
 2.	CAI-2-A     	Delivery Box	Gonville & Caius Colle	2021-03-10 13:07:40.	Covid Testing       
 3.	CAI-2-B     	Delivery Box	None                  	None                	None                
 4.	CAI-3-A     	Delivery Box	Gonville & Caius Colle	2021-03-10 13:07:42.	Covid Testing       
 5.	CAI-3-B     	Delivery Box	Box Storage Room      	2021-03-11 09:46:13.	Covid Testing       
 6.	CC-1-A      	Delivery Box	Corpus Christi Main Pl	2021-03-08 12:41:08.	Covid Testing       
 7.	CC-1-B      	Delivery Box	Box Storage Room      	2021-03-09 11:19:29.	Covid Testing       
 8.	CC-2-A      	Delivery 

In [18]:
df = pd.DataFrame(data)

In [19]:
df.sort_values("last_seen_dt", ascending=False)

,box,collection,location,last_seen_dt,last_seen_by,last_seen_by_email
84,JE-P-1-B,Drop-Off Point Box,Anne McLaren Building,2021-03-12 13:58:31.839000+00:00,Covid Testing,cu.covid.testing.1@gmail.com
104,JN-P-2-B,Drop-Off Point Box,Anne McLaren Building,2021-03-12 13:58:29.437000+00:00,Covid Testing,cu.covid.testing.1@gmail.com
131,N-P-1-B,Drop-Off Point Box,Anne McLaren Building,2021-03-12 13:58:26.244000+00:00,Covid Testing,cu.covid.testing.1@gmail.com
45,ENG-P-1-B,Drop-Off Point Box,Anne McLaren Building,2021-03-12 13:58:24.367000+00:00,Covid Testing,cu.covid.testing.1@gmail.com
50,ENG-P-2-B,Drop-Off Point Box,Anne McLaren Building,2021-03-12 13:58:22.717000+00:00,Covid Testing,cu.covid.testing.1@gmail.com
...,...,...,...,...,...,...
90,JN-0-A,Delivery Box - Red,None,NaT,None,None
91,JN-0-B,Delivery Box - Red,None,NaT,None,None
112,LC-0-A,Delivery Box - Red,None,NaT,None,None
113,LC-0-B,Delivery Box - Red,None,NaT,None,None


In [20]:
df["id"] = df["box"].str[:-2]
df["rota"] = df["box"].str[-1]
df

,box,collection,location,last_seen_dt,last_seen_by,last_seen_by_email,id,rota
0,CAI-1-A,Delivery Box - Red,Gonville & Caius College,2021-03-10 13:07:38.250000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-1,A
1,CAI-1-B,Delivery Box - Red,Box Storage Room,2021-03-11 09:41:30.471000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-1,B
2,CAI-2-A,Delivery Box - Red,Gonville & Caius College,2021-03-10 13:07:40.699000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-2,A
3,CAI-2-B,Delivery Box - Red,None,NaT,None,None,CAI-2,B
4,CAI-3-A,Delivery Box - Red,Gonville & Caius College,2021-03-10 13:07:42.651000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-3,A
...,...,...,...,...,...,...,...,...
190,WY-P-1-C,Delivery Box - Red,Box Storage Room,2021-01-18 13:01:33.723000+00:00,becca Clarke,rjc204@cam.ac.uk,WY-P-1,C
191,WY-P-1-D,Delivery Box - Red,Box Storage Room,2021-01-18 13:01:13.839000+00:00,becca Clarke,rjc204@cam.ac.uk,WY-P-1,D
192,WY-P-1-E,Delivery Box - Red,Box Storage Room,2021-01-13 10:07:56.822000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,WY-P-1,E
193,WY-P-2-A,Drop-Off Point Box,None,2020-11-12 10:48:46.590000+00:00,Rob Glew,rg522@cam.ac.uk,WY-P-2,A


## Map actual location names to schedule location names

In [21]:
df["loc_curr"] = df["location"]
df["loc_curr"].fillna("None")
df["loc_curr"] = df["loc_curr"].map(loc_map).fillna("Colleges")
df

,box,collection,location,last_seen_dt,last_seen_by,last_seen_by_email,id,rota,loc_curr
0,CAI-1-A,Delivery Box - Red,Gonville & Caius College,2021-03-10 13:07:38.250000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-1,A,Colleges
1,CAI-1-B,Delivery Box - Red,Box Storage Room,2021-03-11 09:41:30.471000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-1,B,WFB
2,CAI-2-A,Delivery Box - Red,Gonville & Caius College,2021-03-10 13:07:40.699000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-2,A,Colleges
3,CAI-2-B,Delivery Box - Red,None,NaT,None,None,CAI-2,B,Colleges
4,CAI-3-A,Delivery Box - Red,Gonville & Caius College,2021-03-10 13:07:42.651000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-3,A,Colleges
...,...,...,...,...,...,...,...,...,...
190,WY-P-1-C,Delivery Box - Red,Box Storage Room,2021-01-18 13:01:33.723000+00:00,becca Clarke,rjc204@cam.ac.uk,WY-P-1,C,WFB
191,WY-P-1-D,Delivery Box - Red,Box Storage Room,2021-01-18 13:01:13.839000+00:00,becca Clarke,rjc204@cam.ac.uk,WY-P-1,D,WFB
192,WY-P-1-E,Delivery Box - Red,Box Storage Room,2021-01-13 10:07:56.822000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,WY-P-1,E,WFB
193,WY-P-2-A,Drop-Off Point Box,None,2020-11-12 10:48:46.590000+00:00,Rob Glew,rg522@cam.ac.uk,WY-P-2,A,Colleges


In [52]:
df['date'] = df['last_seen_dt'].dt.date
df['time'] = df['last_seen_dt'].dt.strftime("%H:%M")
df

,box,collection,location,last_seen_dt,last_seen_by,last_seen_by_email,id,rota,loc_curr,date,time
0,CAI-1-A,Delivery Box - Red,Gonville & Caius College,2021-03-10 13:07:38.250000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-1,A,Colleges,2021-03-10,13:07
1,CAI-1-B,Delivery Box - Red,Box Storage Room,2021-03-11 09:41:30.471000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-1,B,WFB,2021-03-11,09:41
2,CAI-2-A,Delivery Box - Red,Gonville & Caius College,2021-03-10 13:07:40.699000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-2,A,Colleges,2021-03-10,13:07
3,CAI-2-B,Delivery Box - Red,None,NaT,None,None,CAI-2,B,Colleges,NaT,NaN
4,CAI-3-A,Delivery Box - Red,Gonville & Caius College,2021-03-10 13:07:42.651000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-3,A,Colleges,2021-03-10,13:07
5,CAI-3-B,Delivery Box - Red,Box Storage Room,2021-03-11 09:46:13.728000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-3,B,WFB,2021-03-11,09:46
6,CC-1-A,Delivery Box - Red,Corpus Christi Main Plodge,2021-03-08 12:41:08.355000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CC-1,A,Colleges,2021-03-08,12:41
7,CC-1-B,Delivery Box - Red,Box Storage Room,2021-03-09 11:19:29.365000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CC-1,B,WFB,2021-03-09,11:19
8,CC-2-A,Delivery Box - Red,Corpus Leckhampton,2021-03-08 13:01:11.265000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CC-2,A,Colleges,2021-03-08,13:01
9,CC-2-B,Delivery Box - Red,Box Storage Room,2021-03-09 11:20:39.572000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CC-2,B,WFB,2021-03-09,11:20


# Excel Schedule

In [53]:
df_boxA = pd.read_excel("UMS_Courier_Schedules.xlsx", sheet_name="Box_Group_Lookup")
df_boxB = df_boxA.copy()

df_boxA["rota"] = "A"
df_boxA["id"] = df_boxA["Box"]
df_boxA["Box"] = df_boxA["Box"] + "-" + df_boxA["rota"]
df_boxA["Group"] = df_boxA["Group"] + "-" + df_boxA["rota"]

df_boxB["rota"] = "B"
df_boxB["id"] = df_boxB["Box"]
df_boxB["Box"] = df_boxB["Box"] + "-" + df_boxB["rota"]
df_boxB["Group"] = df_boxB["Group"] + "-" + df_boxB["rota"]

df_box = pd.concat([df_boxA, df_boxB]).sort_values("Box").set_index("Box", drop=True)

df_box

,Collection,Group,rota,id
Box,,,,
CAI-1-A,Delivery Box - Red,R3-A,A,CAI-1
CAI-1-B,Delivery Box - Red,R3-B,B,CAI-1
CAI-2-A,Delivery Box - Red,R3-A,A,CAI-2
CAI-2-B,Delivery Box - Red,R3-B,B,CAI-2
CAI-3-A,Delivery Box - Red,R3-A,A,CAI-3
CAI-3-B,Delivery Box - Red,R3-B,B,CAI-3
CC-1-A,Delivery Box - Red,R1-A,A,CC-1
CC-1-B,Delivery Box - Red,R1-B,B,CC-1
CC-2-A,Delivery Box - Red,R1-A,A,CC-2


In [54]:
df_sch = pd.read_excel(
    "UMS_Courier_Schedules.xlsx", sheet_name="Group_Schedule", index_col="Group"
)
df_sch

,Monday-A,Tuesday-A,Wednesday-A,Thursday-A,Friday-A,Saturday-A,Sunday-A,Monday-B,Tuesday-B,Wednesday-B,Thursday-B,Friday-B,Saturday-B,Sunday-B
Group,,,,,,,,,,,,,,
R1-A,AMB,WFB,WFB,WFB,WFB,WFB,WFB,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges
R2-A,Colleges,AMB,WFB,WFB,WFB,WFB,WFB,WFB,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges
R3-A,Colleges,Colleges,AMB,WFB,WFB,WFB,WFB,WFB,WFB,Colleges,Colleges,Colleges,Colleges,Colleges
R1-B,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,AMB,WFB,WFB,WFB,WFB,WFB,WFB
R2-B,WFB,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,AMB,WFB,WFB,WFB,WFB,WFB
R3-B,WFB,WFB,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,AMB,WFB,WFB,WFB,WFB
D1-A,AMB,DO,AMB,DO,AMB,AMB,AMB,DO,AMB,DO,AMB,DO,DO,DO
D1-B,DO,AMB,DO,AMB,DO,DO,DO,AMB,DO,AMB,DO,AMB,AMB,AMB
D2-A,AMB,DO,AMB,DO,DO,DO,DO,AMB,DO,AMB,DO,DO,DO,DO


In [55]:
day = "Monday-A"
box = "HO-1-A"

group = df_box.loc[box, "Group"]


df_sch.loc[group, day]

'AMB'

In [56]:
days = df_sch.columns.to_list()
boxes = df_box.index.to_list()

from typing import List


def get_expected_location(boxes: List[str], days: str) -> pd.DataFrame:
    """
    Gets expected location of boxes at the end of days

    Params
    ------
    box: List[str]
        - list of boxes
    day: str
        - day

    Returns
    -------
    pd.DataFrame

    """
    group = df_box.loc[boxes, "Group"]
    df_out = df_sch.loc[group, [days]]
    df_out["Box"] = boxes
    df_out = df_out.set_index("Box", drop=True)
    df_out["Group"] = group
    df_out = df_out.rename(columns={f"{days}": "loc_exp"})
    return df_out


# get_expected_location(box=boxes[0:2], day=days[0:2])
# get_expected_location(boxes=['HO-1-A', 'HO-1-B', 'W-1-A'], days=days[0:2])
get_expected_location(["HO-1-A"], "Sunday-B")

,loc_exp,Group
Box,,
HO-1-A,Colleges,R1-A


In [67]:
# boxes = ['HO-1-B']
day = "Sunday-B"

df_exp = get_expected_location(boxes, day)

df_act = df.loc[df["box"].isin(boxes), :]

df_locations = pd.merge(df_act, df_exp, left_on="box", right_index=True).set_index(
    "box", drop=True
)

df_locations["location_is_correct"] = (
    df_locations["loc_curr"] == df_locations["loc_exp"]
)

df_locations = df_locations[
    [
        "location_is_correct",
        "date",
        "last_seen_by",
        "last_seen_by_email",
        "Group",
        "loc_exp",
        "loc_curr",
        "location",
    ]
]

df_locations = df_locations.sort_values(["location_is_correct", "date", "box"])

[[datetime.date(2021, 2, 25),
  'Covid Testing',
  'cu.covid.testing.1@gmail.com',
  'D2-A',
  'DO',
  'AMB',
  'Anne McLaren Building'],
 [datetime.date(2021, 3, 8),
  'Covid Testing',
  'cu.covid.testing.4@gmail.com',
  'R1-A',
  'Colleges',
  'UMS',
  'In Transit'],
 [datetime.date(2021, 3, 8),
  'Covid Testing',
  'cu.covid.testing.4@gmail.com',
  'R1-A',
  'Colleges',
  'UMS',
  'In Transit'],
 [datetime.date(2021, 3, 11),
  'Covid Testing',
  'cu.covid.testing.4@gmail.com',
  'D2-B',
  'AMB',
  'UMS',
  'In-Transit Paul'],
 [datetime.date(2021, 3, 11),
  'Covid Testing',
  'cu.covid.testing.1@gmail.com',
  'R2-B',
  'WFB',
  'AMB',
  'Anne McLaren Building'],
 [NaT, None, None, 'R3-B', 'WFB', 'Colleges', None]]

In [61]:
pd.set_option("display.max_rows", None)
df_locations.sort_values(
    by=["location_is_correct", "date", "loc_curr", "location"],
    ascending=[True, False, True, True],
)
# pd.reset_option("display.max_rows")

,location_is_correct,date,last_seen_by,last_seen_by_email,Group,loc_exp,loc_curr,location
box,,,,,,,,
TH-1-B,False,2021-03-11,Covid Testing,cu.covid.testing.1@gmail.com,R2-B,WFB,AMB,Anne McLaren Building
G-P-1-B,False,2021-03-11,Covid Testing,cu.covid.testing.4@gmail.com,D2-B,AMB,UMS,In-Transit Paul
DOW-1-A,False,2021-03-08,Covid Testing,cu.covid.testing.4@gmail.com,R1-A,Colleges,UMS,In Transit
DOW-2-A,False,2021-03-08,Covid Testing,cu.covid.testing.4@gmail.com,R1-A,Colleges,UMS,In Transit
G-P-1-A,False,2021-02-25,Covid Testing,cu.covid.testing.1@gmail.com,D2-A,DO,AMB,Anne McLaren Building
CAI-2-B,False,NaT,None,None,R3-B,WFB,Colleges,None
ENG-P-1-B,True,2021-03-12,Covid Testing,cu.covid.testing.1@gmail.com,D1-B,AMB,AMB,Anne McLaren Building
ENG-P-2-B,True,2021-03-12,Covid Testing,cu.covid.testing.1@gmail.com,D1-B,AMB,AMB,Anne McLaren Building
JE-P-1-B,True,2021-03-12,Covid Testing,cu.covid.testing.1@gmail.com,D1-B,AMB,AMB,Anne McLaren Building
